### Solver API scratchbook

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import time

from nuMSM_solver.solvers import *
from nuMSM_solver.quadrature import GaussianQuadrature
from nuMSM_solver.rates import Rates_Jurai

from leptotools.leptoAPI import LeptoToolsSolver

In [2]:
# Optimal phases
delta_opt = np.pi
eta_opt = (3.0*np.pi)/2.0
rew_opt = np.pi/4.0

# Model params
mp = ModelParams(M=1.0, dM=1e-11, Imw=3, Rew=rew_opt, delta=delta_opt, eta=eta_opt)

# Metaparams
H = 1
ode_pars = {'atol': 1e-20, 'rtol': 1e-4}

In [3]:
# Set up quadrature scheme and rates
n_kc = 5
kc_max = 10

#TODO factor out rates from quadrature class, no idea why I did it like that

# Hide annoying warning
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    quad = GaussianQuadrature(n_kc, 0, kc_max, mp, H, qscheme="legendre")
    rates = Rates_Jurai(mp, H, quad.kc_list(), tot=False)

In [ ]:
# Solve equations with QuadratureSolver
quad_solver = QuadratureSolver(rates, quad,
                         model_params=mp, TF=Tsph, H=H, cutoff=1e-5, eig_cutoff=False,
                         method="LSODE", ode_pars=ode_pars, source_term=True)

start = time.time()
quad_solver.solve()
end = time.time()
quad_bau = (28./79.) * quad_solver.get_final_lepton_asymmetry()
print("QuadratureSolver; time (solve): {}, BAU = {}".format(end - start, quad_bau))

In [4]:
# Solve equations with LeptoToolsSolver
lepto_solver = LeptoToolsSolver(model_params=mp, TF=Tsph, quadrature=quad, H=H, cutoff=1e-5, 
                          method="LSODE", ode_pars=ode_pars)

start = time.time()
lepto_solver.solve()
end = time.time()
lepto_bau = (28./79.) * lepto_solver.get_final_lepton_asymmetry()
print("LeptoToolsSolver; time (solve): {}, BAU = {}".format(end - start, lepto_bau))

/home/admin/SciCodes/leptogenesis-tools/leptotools/momentumDep.py:46: RuntimeWarning: divide by zero encountered in log
  lgM=np.log(gammaMFromQm(M, kT_ax, T_ax, Qm_F)/T_ax)[argK]


m:  [0.00753087 0.00753087 0.00759301 0.00759301 0.00759301 0.00759301
 0.00759301 0.00759301 0.00757836 0.00757836 0.00759301 0.00759301
 0.00759301 0.00759301 0.00759301 0.00759301 0.00759193 0.00759193
 0.00759301 0.00759301 0.00759301 0.00759301 0.00759301 0.00759301
 0.00759294 0.00759294 0.00759301 0.00759301 0.00759301 0.00759301
 0.00759301 0.00759301 0.007593   0.007593   0.00759301 0.00759301
 0.00759301 0.00759301 0.00759301 0.00759301 0.00759301 0.00759301
 0.00759301]
ls.Mb:  1.0
Tsph 131.7
zeta [-8.21831523e-03 -8.21831523e-03 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -1.93250528e-03 -1.93250528e-03 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -1.42930871e-04 -1.42930871e-04 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -9.74031727e-06 -9.74031727e-06 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+0

ValueError: operands could not be broadcast together with shapes (43,) (5,) 

In [ ]:
# Plot BAU generation
fig = plt.figure(dpi=60)
ax = fig.add_subplot(111)
plt.xlabel("T")
plt.ylabel("bau")
plt.xscale("log")
plt.yscale("log")
plt.plot(solver.get_Tlist(), (28./79.) * np.abs(np.array(solver.get_total_lepton_asymmetry())))